# Rookie's Data

We need to load and clean the data from the draft classes to predict their fantasy scores. 

In [1]:
# imports
import pandas as pd
import numpy as np  

# load the data
draft2020 = pd.read_csv('newdata/2020draftraw.csv', header=[0,1])
draft2021 = pd.read_csv('newdata/2021draftraw.csv', header=[0,1])
draft2022 = pd.read_csv('newdata/2022draftraw.csv', header=[0,1])
draft2023 = pd.read_csv('newdata/2023draftraw.csv', header=[0,1])
draft2024 = pd.read_csv('newdata/2024draftraw.csv', header=[0,1])
draft2025 = pd.read_csv('newdata/2025draftraw.csv', header=[0,1])

# add the year column
draft2020['Draft_Year'] = 2020
draft2021['Draft_Year'] = 2021
draft2022['Draft_Year'] = 2022
draft2023['Draft_Year'] = 2023
draft2024['Draft_Year'] = 2024
draft2025['Draft_Year'] = 2025

# combine the data
drafts_raw = pd.concat([draft2020, draft2021, draft2022, draft2023, draft2024, draft2025], ignore_index=True)
drafts_raw.head()

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Unnamed: 3_level_0  \
                 Rnd               Pick                 Tm             Player   
0                  1                  1                CIN         Joe Burrow   
1                  1                  2                WAS        Chase Young   
2                  1                  3                DET        Jeff Okudah   
3                  1                  4                NYG      Andrew Thomas   
4                  1                  5                MIA     Tua Tagovailoa   

  Unnamed: 4_level_0 Unnamed: 5_level_0 Unnamed: 6_level_0 Misc     \
                 Pos                Age                 To  AP1 PB   
0                 QB               23.0             2025.0    0  2   
1                 DE               21.0             2024.0    0  1   
2                 CB               21.0             2025.0    0  0   
3                  T               21.0             2025.0    0  0   
4                 QB               22.0             2025.0    0  1   

  Unnamed: 9_level_0  ... Receiving           Unnamed: 24_level_0  \
                  St  ...       Rec  Yds   TD                Solo   
0                  5  ...       0.0  0.0  0.0                 1.0   
1                  1  ...       0.0  0.0  0.0                88.0   
2                  2  ...       0.0  0.0  0.0               150.0   
3                  5  ...       1.0  2.0  1.0                 7.0   
4                  5  ...       0.0  0.0  0.0                 NaN   

  Unnamed: 25_level_0 Unnamed: 26_level_0 Unnamed: 27_level_0  \
                  Int                  Sk        College/Univ   
0                 NaN                 NaN                 LSU   
1                 NaN                22.0            Ohio St.   
2                 2.0                 NaN            Ohio St.   
3                 NaN                 NaN             Georgia   
4                 NaN                 NaN             Alabama   

  Unnamed: 28_level_0 -additional Draft_Year  
  Unnamed: 28_level_1       -9999             
0       College Stats    BurrJo01       2020  
1       College Stats    YounCh04       2020  
2       College Stats    OkudJe00       2020  
3       College Stats    ThomAn02       2020  
4       College Stats    TagoTu00       2020  

[5 rows x 31 columns]

### Fix the positions

The draft is for real NFL teams to pick players that they need in every position. Not all of these positions are fantasy relevant, so we are only going to look at the players in the following positions: WR, QB, RB, TE, FB.

In [2]:
# select only relevant positions
drafts_raw = drafts_raw[drafts_raw[('Unnamed: 4_level_0', 'Pos')].isin(['WR', 'QB', 'RB', 'TE', 'FB'])]

#see the distinct positions
drafts_raw[('Unnamed: 4_level_0', 'Pos')].unique()

array(['QB', 'WR', 'RB', 'TE', 'FB'], dtype=object)

#### Clean the column names

In [3]:
drafts_raw.columns = ['_'.join(col).strip() for col in drafts_raw.columns.values]

# drop underscores that are not needed
drafts_raw.columns = [c.replace('__', '_').replace('-', '').strip('_') for c in drafts_raw.columns]

# check column names
drafts_raw.columns

# rename some columns
drafts_raw = drafts_raw.rename(columns={"Unnamed: 0_level_0_Rnd": "Round",
                                      "Unnamed: 1_level_0_Pick": "Pick",
                                        "Unnamed: 2_level_0_Tm": "Team",
                                        "Unnamed: 3_level_0_Player": "Player",
                                        "Unnamed: 4_level_0_Pos": "Position",
                                        "Unnamed: 5_level_0_Age": "Age",
                                        "Unnamed: 27_level_0_College/Univ": "College"})
drafts_raw.columns
                                      

Index(['Round', 'Pick', 'Team', 'Player', 'Position', 'Age',
       'Unnamed: 6_level_0_To', 'Misc_AP1', 'Misc_PB', 'Unnamed: 9_level_0_St',
       'Approx Val_wAV', 'Approx Val_DrAV', 'Unnamed: 12_level_0_G',
       'Passing_Cmp', 'Passing_Att', 'Passing_Yds', 'Passing_TD',
       'Passing_Int', 'Rushing_Att', 'Rushing_Yds', 'Rushing_TD',
       'Receiving_Rec', 'Receiving_Yds', 'Receiving_TD',
       'Unnamed: 24_level_0_Solo', 'Unnamed: 25_level_0_Int',
       'Unnamed: 26_level_0_Sk', 'College',
       'Unnamed: 28_level_0_Unnamed: 28_level_1', 'additional_9999',
       'Draft_Year'],
      dtype='object')

#### Now we can select only the data that we need

In [4]:
#select only the data that we need
drafts_raw = drafts_raw[['Draft_Year', 'Round', 'Pick', 'Team', 'Player', 'Position', 'Age', 'College']]

# strip *+ from player names
drafts_raw['Player'] = drafts_raw['Player'].str.replace(r'[\*\+]', '', regex=True)

#rename any colleges to match combined stats data
college_rename_dict = {'Texas Christian': 'TCU',
                       'Appalachian St.': 'Appalachian State',
                        'Arizona St.': 'Arizona State',
                        'Boise St.': 'Boise State',
                        'Central Missouri St.': 'Central Missouri',
                        'Colorado St.': 'Colorado State',
                        'Florida St.': 'Florida State',
                        'Fresno St.': 'Fresno State',
                        'Iowa St.': 'Iowa State',
                        'Kansas St.': 'Kansas State',
                        'Michigan St.': 'Michigan State',
                        'Mississippi St.': 'Mississippi State',
                        'Montana St.': 'Montana State',
                        'New Mexico St.': 'New Mexico State',
                        'North Dakota St.': 'North Dakota State',
                        'Ohio St.': 'Ohio State',
                        'Oklahoma St.': 'Oklahoma State',
                        'Oregon St.': 'Oregon State',
                        'Penn St.': 'Penn State',
                        'SE Missouri St.': 'Southeast Missouri State',
                        'San Diego St.': 'San Diego State',
                        'South Dakota St.': 'South Dakota State',
                        'Utah St.': 'Utah State',
                        'Washington St.': 'Washington State',
                        'Youngstown St.': 'Youngstown State',
                        'Boston Col.': 'Boston College',
                        'Central Florida': 'UCF',
                        'Southern Miss': 'Southern Mississippi',
                        'Ala-Birmingham': 'UAB',}
drafts_raw['College'] = drafts_raw['College'].replace(college_rename_dict)

# create a dual key with name and year
#drafts_raw = drafts_raw.set_index(['Player','Draft_Year'])
drafts_raw.head()

,Draft_Year,Round,Pick,Team,Player,Position,Age,College
0,2020,1,1,CIN,Joe Burrow,QB,23.0,LSU
4,2020,1,5,MIA,Tua Tagovailoa,QB,22.0,Alabama
5,2020,1,6,LAC,Justin Herbert,QB,22.0,Oregon
11,2020,1,12,LVR,Henry Ruggs III,WR,21.0,Alabama
14,2020,1,15,DEN,Jerry Jeudy,WR,21.0,Alabama


Now we have finished cleaning the data for the drafts, now we can work on cleaning their respective college stats from the year before they were drafted. 

# Clean College Stats

In [5]:
# load and combine the data by stat type
def load_and_combine_data(stat_type):
    df_list = []
    for year in range(2019, 2025):
        if stat_type != "passing":
            df = pd.read_csv(f'college stats/{year}_{stat_type}.csv', header=[0,1])
        else:
            df = pd.read_csv(f'college stats/{year}_passing.csv')
        df['Draft_Year'] = year + 1 # add 1 to align with draft year
        df_list.append(df)
    combined_df = pd.concat(df_list, ignore_index=True)
    return combined_df

# load the data
passing_raw, rushing_raw, receiving_raw = [load_and_combine_data(stat) for stat in ['passing', 'rushing', 'receiving']]

passing_raw.head()

,Rk,Player,Team,Conf,G,Cmp,Att,Cmp%,Yds,TD,...,Int,Int%,Y/A,AY/A,Y/C,Y/G,Rate,Awards,Player-additional,Draft_Year
0,1,Joe Burrow*,LSU,SEC,15,402,527,76.3,5671,60,...,6,1.1,10.8,12.53,14.1,378.1,202.0,H-1MaxwellAACamp,joe-burrow-1,2020
1,2,Anthony Gordon*,Washington State,Pac-12,13,493,689,71.6,5579,48,...,16,2.3,8.1,8.45,11.3,429.2,157.9,NaN,anthony-gordon-2,2020
2,3,Cole McDonald*,Hawaii,MWC,14,326,511,63.8,4135,33,...,14,2.7,8.1,8.15,12.7,295.4,147.6,NaN,cole-mcdonald-1,2020
3,4,Brady White*,Memphis,American,14,269,420,64.0,4014,33,...,11,2.6,9.6,9.95,14.9,286.7,165.0,NaN,brady-white-1,2020
4,5,Brock Purdy*,Iowa State,Big 12,13,312,475,65.7,3982,27,...,9,1.9,8.4,8.67,12.8,306.3,151.1,NaN,brock-purdy-1,2020


In [6]:
rushing_raw.head()

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Unnamed: 3_level_0  \
                  Rk             Player               Team               Conf   
0                  1     Chuba Hubbard*     Oklahoma State             Big 12   
1                  2     Malcolm Perry*               Navy           American   
2                  3      J.K. Dobbins*         Ohio State            Big Ten   
3                  4   Jonathan Taylor*          Wisconsin            Big Ten   
4                  5   Jaret Patterson*            Buffalo                MAC   

  Unnamed: 4_level_0 Rushing                        ... Receiving           \
                   G     Att   Yds  Y/A  TD    Y/G  ...       Y/R TD   Y/G   
0                 13     328  2094  6.4  21  161.1  ...       8.6  0  15.2   
1                 13     295  2017  6.8  21  155.2  ...       NaN  0   0.0   
2                 14     301  2003  6.7  21  143.1  ...      10.7  2  17.6   
3                 14     320  2003  6.3  21  143.1  ...       9.7  5  18.0   
4                 13     312  1799  5.8  19  138.4  ...      16.1  1  16.1   

  Scrimmage                Unnamed: 19_level_0        -additional Draft_Year  
      Plays   Yds  Avg  TD              Awards              -9999             
0       351  2292  6.5  21               H-8AA    chuba-hubbard-1       2020  
1       295  2017  6.8  21                 NaN    malcolm-perry-1       2020  
2       324  2250  6.9  23                 H-6       jk-dobbins-1       2020  
3       346  2255  6.5  26               H-5AA  jonathan-taylor-1       2020  
4       325  2008  6.2  20                 NaN  jaret-patterson-1       2020  

[5 rows x 22 columns]

In [7]:
receiving_raw.head()

Unnamed: 0_level_0     Unnamed: 1_level_0 Unnamed: 2_level_0  \
                  Rk                 Player               Team   
0                  1         Ja'Marr Chase*                LSU   
1                  2          Omar Bayless*     Arkansas State   
2                  3      Justin Jefferson*                LSU   
3                  4  Antonio Gandy-Golden*            Liberty   
4                  5        Devin Duvernay*              Texas   

  Unnamed: 3_level_0 Unnamed: 4_level_0 Receiving                         ...  \
                Conf                  G       Rec   Yds   Y/R  TD    Y/G  ...   
0                SEC                 14        84  1780  21.2  20  127.1  ...   
1           Sun Belt                 13        93  1653  17.8  17  127.2  ...   
2                SEC                 15       111  1540  13.9  18  102.7  ...   
3                Ind                 13        79  1396  17.7  10  107.4  ...   
4             Big 12                 13       106  1386  13.1   9  106.6  ...   

  Rushing         Scrimmage                 Unnamed: 19_level_0  \
      Y/A TD  Y/G     Plays   Yds   Avg  TD              Awards   
0     5.0  0  0.4        85  1785  21.0  20                  AA   
1     NaN  0  0.0        93  1653  17.8  17                 NaN   
2     NaN  0  0.0       111  1540  13.9  18                 NaN   
3     NaN  0  0.0        79  1396  17.7  10                 NaN   
4     2.4  1  1.8       116  1410  12.2  10                 NaN   

              -additional Draft_Year  
                    -9999             
0          jamarr-chase-1       2020  
1          omar-bayless-1       2020  
2      justin-jefferson-1       2020  
3  antonio-gandy-golden-1       2020  
4        devin-duvernay-1       2020  

[5 rows x 22 columns]

#### Fix Column names and row entries

In [8]:
# check column names for passing
passing_raw.columns

Index(['Rk', 'Player', 'Team', 'Conf', 'G', 'Cmp', 'Att', 'Cmp%', 'Yds', 'TD',
       'TD%', 'Int', 'Int%', 'Y/A', 'AY/A', 'Y/C', 'Y/G', 'Rate', 'Awards',
       'Player-additional', 'Draft_Year'],
      dtype='object')

In [9]:
#remove some columns for passing
passing_raw = passing_raw.drop(columns = ["Rk","Cmp%","TD%","Y/A","AY/A","Y/C","Y/G","Rate","Awards","Player-additional","Int%"])
passing_raw.columns

Index(['Player', 'Team', 'Conf', 'G', 'Cmp', 'Att', 'Yds', 'TD', 'Int',
       'Draft_Year'],
      dtype='object')

In [10]:
#rename some columns for passing
passing_raw = passing_raw.rename(columns={"Att": "Pass_Att",
                                          "Yds": "Pass_Yds",
                                            "TD": "Pass_TD"})

# strip * from player names
passing_raw['Player'] = passing_raw['Player'].str.replace(r'[\*\+]', '', regex=True)
passing_raw.head()

,Player,Team,Conf,G,Cmp,Pass_Att,Pass_Yds,Pass_TD,Int,Draft_Year
0,Joe Burrow,LSU,SEC,15,402,527,5671,60,6,2020
1,Anthony Gordon,Washington State,Pac-12,13,493,689,5579,48,16,2020
2,Cole McDonald,Hawaii,MWC,14,326,511,4135,33,14,2020
3,Brady White,Memphis,American,14,269,420,4014,33,11,2020
4,Brock Purdy,Iowa State,Big 12,13,312,475,3982,27,9,2020


In [11]:
# create a dual key with name and year
#passing_raw = passing_raw.set_index(['Player','Draft_Year'])
passing_raw.head()

,Player,Team,Conf,G,Cmp,Pass_Att,Pass_Yds,Pass_TD,Int,Draft_Year
0,Joe Burrow,LSU,SEC,15,402,527,5671,60,6,2020
1,Anthony Gordon,Washington State,Pac-12,13,493,689,5579,48,16,2020
2,Cole McDonald,Hawaii,MWC,14,326,511,4135,33,14,2020
3,Brady White,Memphis,American,14,269,420,4014,33,11,2020
4,Brock Purdy,Iowa State,Big 12,13,312,475,3982,27,9,2020


In [12]:
# move onto rushing
# combine both headers into one
rushing_raw.columns = ['_'.join(col).strip() for col in rushing_raw.columns.values]

rushing_raw.columns

Index(['Unnamed: 0_level_0_Rk', 'Unnamed: 1_level_0_Player',
       'Unnamed: 2_level_0_Team', 'Unnamed: 3_level_0_Conf',
       'Unnamed: 4_level_0_G', 'Rushing_Att', 'Rushing_Yds', 'Rushing_Y/A',
       'Rushing_TD', 'Rushing_Y/G', 'Receiving_Rec', 'Receiving_Yds',
       'Receiving_Y/R', 'Receiving_TD', 'Receiving_Y/G', 'Scrimmage_Plays',
       'Scrimmage_Yds', 'Scrimmage_Avg', 'Scrimmage_TD',
       'Unnamed: 19_level_0_Awards', '-additional_-9999', 'Draft_Year_'],
      dtype='object')

In [13]:
#rename some columns
rushing_raw = rushing_raw.rename(columns={"Unnamed: 1_level_0_Player": "Player",
                                          "Unnamed: 2_level_0_Team": "Team",
                                          "Unnamed: 3_level_0_Conf": "Conf",
                                           "Unnamed: 4_level_0_G": "G",
                                           "Draft_Year_": "Draft_Year"})


# select only the relevant columns
rushing_raw = rushing_raw[['Player','Team','Conf', 'Draft_Year','G', 'Rushing_Att', 'Rushing_Yds', 'Rushing_TD']]
rushing_raw.head()

,Player,Team,Conf,Draft_Year,G,Rushing_Att,Rushing_Yds,Rushing_TD
0,Chuba Hubbard*,Oklahoma State,Big 12,2020,13,328,2094,21
1,Malcolm Perry*,Navy,American,2020,13,295,2017,21
2,J.K. Dobbins*,Ohio State,Big Ten,2020,14,301,2003,21
3,Jonathan Taylor*,Wisconsin,Big Ten,2020,14,320,2003,21
4,Jaret Patterson*,Buffalo,MAC,2020,13,312,1799,19


In [14]:
# remove * from player names
rushing_raw['Player'] = rushing_raw['Player'].str.replace(r'[\*\+]', '', regex=True)

# create a dual key with name and year
#rushing_raw = rushing_raw.set_index(['Player','Draft_Year'])
rushing_raw.head()

,Player,Team,Conf,Draft_Year,G,Rushing_Att,Rushing_Yds,Rushing_TD
0,Chuba Hubbard,Oklahoma State,Big 12,2020,13,328,2094,21
1,Malcolm Perry,Navy,American,2020,13,295,2017,21
2,J.K. Dobbins,Ohio State,Big Ten,2020,14,301,2003,21
3,Jonathan Taylor,Wisconsin,Big Ten,2020,14,320,2003,21
4,Jaret Patterson,Buffalo,MAC,2020,13,312,1799,19


In [15]:
# now for receiving
# combine both headers into one
receiving_raw.columns = ['_'.join(col).strip() for col in receiving_raw.columns.values]
receiving_raw.columns

Index(['Unnamed: 0_level_0_Rk', 'Unnamed: 1_level_0_Player',
       'Unnamed: 2_level_0_Team', 'Unnamed: 3_level_0_Conf',
       'Unnamed: 4_level_0_G', 'Receiving_Rec', 'Receiving_Yds',
       'Receiving_Y/R', 'Receiving_TD', 'Receiving_Y/G', 'Rushing_Att',
       'Rushing_Yds', 'Rushing_Y/A', 'Rushing_TD', 'Rushing_Y/G',
       'Scrimmage_Plays', 'Scrimmage_Yds', 'Scrimmage_Avg', 'Scrimmage_TD',
       'Unnamed: 19_level_0_Awards', '-additional_-9999', 'Draft_Year_'],
      dtype='object')

In [16]:
# rename some columns for receiving
receiving_raw = receiving_raw.rename(columns={"Unnamed: 1_level_0_Player": "Player",
                                              "Unnamed: 2_level_0_Team": "Team",
                                                "Unnamed: 3_level_0_Conf": "Conf",
                                                "Unnamed: 4_level_0_G": "G",
                                                "Draft_Year_": "Draft_Year",
                                                "Receiving_Rec":"Rec"})

# select only the relevant columns
receiving_raw = receiving_raw[['Player','Team','Conf', 'Draft_Year', 'G','Rec', 'Receiving_Yds', 'Receiving_TD']]
receiving_raw.head()

,Player,Team,Conf,Draft_Year,G,Rec,Receiving_Yds,Receiving_TD
0,Ja'Marr Chase*,LSU,SEC,2020,14,84,1780,20
1,Omar Bayless*,Arkansas State,Sun Belt,2020,13,93,1653,17
2,Justin Jefferson*,LSU,SEC,2020,15,111,1540,18
3,Antonio Gandy-Golden*,Liberty,Ind,2020,13,79,1396,10
4,Devin Duvernay*,Texas,Big 12,2020,13,106,1386,9


In [17]:
# strip * from player names
receiving_raw['Player'] = receiving_raw['Player'].str.replace(r'[\*\+]', '', regex=True)

# create a dual key with name and year
#receiving_raw = receiving_raw.set_index(['Player','Draft_Year'])
receiving_raw.head()

,Player,Team,Conf,Draft_Year,G,Rec,Receiving_Yds,Receiving_TD
0,Ja'Marr Chase,LSU,SEC,2020,14,84,1780,20
1,Omar Bayless,Arkansas State,Sun Belt,2020,13,93,1653,17
2,Justin Jefferson,LSU,SEC,2020,15,111,1540,18
3,Antonio Gandy-Golden,Liberty,Ind,2020,13,79,1396,10
4,Devin Duvernay,Texas,Big 12,2020,13,106,1386,9


#### Now we will need to combine all of the three categories into one big dataset

In [18]:
# combine all of the three categories into one big dataset
# will join on Player and Draft_Year and also Team, Conf, and G to ensure accuracy
# Use outer joins to ensure no data is lost
combined_stats = pd.merge(passing_raw, rushing_raw, how='outer', left_on=['Player','Draft_Year','Team','Conf','G'], right_on=['Player','Draft_Year','Team','Conf','G'], suffixes=('_pass', '_rush'))

# now merge with receiving
combined_stats = pd.merge(combined_stats, receiving_raw, how='outer', left_on=['Player','Draft_Year','Team','Conf','G'], right_on=['Player','Draft_Year','Team','Conf','G'])

# fill in the NaN values with 0
combined_stats = combined_stats.fillna(0)

#rename team as college to match draft data and draft year to year
combined_stats = combined_stats.rename(columns={"Team": "College"})

#add an extra year to draft year for some players
#Ja'Marr Chase from LSU drafted in 2021 but played 2019 season, so add 1 to his draft year to match
drafts_raw.loc[drafts_raw['Player'].str.lower().str.contains("ja'marr chase"), 'Draft_Year'] = \
    combined_stats.loc[combined_stats['Player'].str.lower().str.contains("ja'marr chase"), 'Draft_Year'].values[0]

drafts_raw.loc[drafts_raw['Player'].str.lower().str.contains("nico collins"), 'Draft_Year'] = \
    combined_stats.loc[combined_stats['Player'].str.lower().str.contains("nico collins"), 'Draft_Year'].values[0]

drafts_raw.loc[drafts_raw['Player'].str.lower().str.contains("kenny gainwell"), 'Draft_Year'] = \
    combined_stats.loc[combined_stats['Player'].str.lower().str.contains("kenny gainwell"), 'Draft_Year'].values[0]


combined_stats.head()


,Player,College,Conf,G,Cmp,Pass_Att,Pass_Yds,Pass_TD,Int,Draft_Year,Rushing_Att,Rushing_Yds,Rushing_TD,Rec,Receiving_Yds,Receiving_TD
0,A'Jon Vivens,Colorado State,MWC,4,0.0,0.0,0.0,0.0,0.0,2021,49.0,210.0,1.0,1.0,1.0,0.0
1,A'Jon Vivens,Colorado State,MWC,9,0.0,0.0,0.0,0.0,0.0,2022,90.0,324.0,0.0,10.0,73.0,0.0
2,A'Jon Vivens,Colorado State,MWC,5,0.0,0.0,0.0,0.0,0.0,2023,39.0,104.0,0.0,4.0,26.0,0.0
3,A'Marion Peterson,USC,Pac-12,2,0.0,0.0,0.0,0.0,0.0,2024,2.0,14.0,0.0,0.0,0.0,0.0
4,A'Marion Peterson,USC,Big Ten,10,0.0,0.0,0.0,0.0,0.0,2025,16.0,65.0,1.0,2.0,13.0,0.0


In [19]:
# Before merging, we should rename some of the colleges to match the draft
# rename some colleges to match draft data
college_rename_dict = {
    'Pitt': 'Pittsburgh',
    'Texas Christian': 'TCU',
    'Ole Miss': 'Mississippi',
    'Nevada-Las Vegas': 'UNLV'
}

combined_stats['College'] = combined_stats['College'].replace(college_rename_dict)

# check for any missing values
combined_stats.isnull().sum()
#check if there are 0s in college, conf, or G
combined_stats[(combined_stats['College'] == 0) | (combined_stats['Conf'] == 0) | (combined_stats['G'] == 0)]

,Player,College,Conf,G,Cmp,Pass_Att,Pass_Yds,Pass_TD,Int,Draft_Year,Rushing_Att,Rushing_Yds,Rushing_TD,Rec,Receiving_Yds,Receiving_TD


### Merge with the draft data

Now we have finished combining all the college stats, we can now merge these stats to the players that actually got drafted
We can merge on the key (Name, Draft Year), which will make sure we get the most recent year before the players were drafted

In [20]:
# merge with the draft data
left = drafts_raw.assign(
    Player_key = drafts_raw['Player'].str.lower()
        .str.replace(r',?\s*\b(jr|sr|ii|iii|iv|v)\.?$', '', regex=True)
        .str.replace(r'[^\w\s]', '', regex=True)   # drop punctuation like periods/commas/apostrophes
        .str.replace(r'\s+', ' ', regex=True)      # collapse spaces
        .str.strip(),
    College_key = drafts_raw['College'].str.lower().str.strip()
)

right = combined_stats.assign(
    Player_key = combined_stats['Player'].str.lower()
        .str.replace(r',?\s*\b(jr|sr|ii|iii|iv|v)\.?$', '', regex=True)
        .str.replace(r'[^\w\s]', '', regex=True)
        .str.replace(r'\s+', ' ', regex=True)
        .str.strip(),
    College_key = combined_stats['College'].str.lower().str.strip()
)

final_df = pd.merge(
    left,
    right,
    how='left',
    left_on=['Player_key','Draft_Year','College_key'],
    right_on=['Player_key','Draft_Year','College_key'],
    suffixes=('_draft','_stats')
)

#remove teh extra key columns
final_df = final_df.drop(columns=['Player_key','College_key','Player_stats','College_stats'])

final_df.head()

,Draft_Year,Round,Pick,Team,Player_draft,Position,Age,College_draft,Conf,G,...,Pass_Att,Pass_Yds,Pass_TD,Int,Rushing_Att,Rushing_Yds,Rushing_TD,Rec,Receiving_Yds,Receiving_TD
0,2020,1,1,CIN,Joe Burrow,QB,23.0,LSU,SEC,15.0,...,527.0,5671.0,60.0,6.0,115.0,368.0,5.0,1.0,16.0,0.0
1,2020,1,5,MIA,Tua Tagovailoa,QB,22.0,Alabama,SEC,9.0,...,252.0,2840.0,33.0,3.0,23.0,17.0,2.0,0.0,0.0,0.0
2,2020,1,6,LAC,Justin Herbert,QB,22.0,Oregon,Pac-12,14.0,...,428.0,3471.0,32.0,6.0,58.0,50.0,4.0,0.0,0.0,0.0
3,2020,1,12,LVR,Henry Ruggs III,WR,21.0,Alabama,SEC,12.0,...,0.0,0.0,0.0,0.0,2.0,75.0,1.0,40.0,746.0,7.0
4,2020,1,15,DEN,Jerry Jeudy,WR,21.0,Alabama,SEC,13.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,77.0,1163.0,10.0


In [21]:
# check for any missing values
final_df.isnull().sum()




Draft_Year        0
Round             0
Pick              0
Team              0
Player_draft      0
Position          0
Age               2
College_draft     0
Conf             22
G                22
Cmp              22
Pass_Att         22
Pass_Yds         22
Pass_TD          22
Int              22
Rushing_Att      22
Rushing_Yds      22
Rushing_TD       22
Rec              22
Receiving_Yds    22
Receiving_TD     22
dtype: int64

In [22]:
# see where the nulls are coming from
final_df[final_df['Pass_TD'].isnull()]

,Draft_Year,Round,Pick,Team,Player_draft,Position,Age,College_draft,Conf,G,...,Pass_Att,Pass_Yds,Pass_TD,Int,Rushing_Att,Rushing_Yds,Rushing_TD,Rec,Receiving_Yds,Receiving_TD
35,2020,3,105,NOR,Adam Trautman,TE,23.0,Dayton,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,2020,5,171,HOU,Isaiah Coulter,WR,21.0,Rhode Island,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,2020,7,231,DAL,Ben DiNucci,QB,23.0,James Madison,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,2021,1,3,SFO,Trey Lance,QB,21.0,North Dakota State,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,2021,5,168,MIN,Zach Davidson,TE,23.0,Central Missouri,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,2021,7,229,IND,Mike Strachan,WR,24.0,Charleston (WV),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161,2022,2,34,GNB,Christian Watson,WR,23.0,North Dakota State,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191,2022,4,127,NWE,Pierre Strong,RB,23.0,South Dakota State,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206,2022,5,162,DEN,Montrell Washington,WR,23.0,Samford,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214,2022,6,192,IND,Andrew Ogletree,TE,24.0,Youngstown State,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Remove NAs in the Data

After looking through the dataset, and adjusting any players that go by nicknames. The players above simply do not have collegiate statistics available on Football Reference. So, since we don't have any stats it should be safe to remove them.  

In [23]:
# remove the NAs in the data
final_rookies = final_df.dropna()

#set aside the 2025 rookies for future work
final_2025_rookies = final_df[final_df['Draft_Year'] == 2025]
final_rookies = final_rookies[final_rookies['Draft_Year'] != 2025]

#save the final data
final_rookies.to_csv('clean_data/final_rookies.csv', index=False)
final_2025_rookies.to_csv('clean_data/final_2025_rookies.csv', index=False)

final_rookies.head()


,Draft_Year,Round,Pick,Team,Player_draft,Position,Age,College_draft,Conf,G,...,Pass_Att,Pass_Yds,Pass_TD,Int,Rushing_Att,Rushing_Yds,Rushing_TD,Rec,Receiving_Yds,Receiving_TD
0,2020,1,1,CIN,Joe Burrow,QB,23.0,LSU,SEC,15.0,...,527.0,5671.0,60.0,6.0,115.0,368.0,5.0,1.0,16.0,0.0
1,2020,1,5,MIA,Tua Tagovailoa,QB,22.0,Alabama,SEC,9.0,...,252.0,2840.0,33.0,3.0,23.0,17.0,2.0,0.0,0.0,0.0
2,2020,1,6,LAC,Justin Herbert,QB,22.0,Oregon,Pac-12,14.0,...,428.0,3471.0,32.0,6.0,58.0,50.0,4.0,0.0,0.0,0.0
3,2020,1,12,LVR,Henry Ruggs III,WR,21.0,Alabama,SEC,12.0,...,0.0,0.0,0.0,0.0,2.0,75.0,1.0,40.0,746.0,7.0
4,2020,1,15,DEN,Jerry Jeudy,WR,21.0,Alabama,SEC,13.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,77.0,1163.0,10.0
